# Исследования уровня NPS телекоммуникационной компании


Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России. 
Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».
Компания провела опрос и попросила вас подготовить дашборд с его итогами.
Чтобы оценить результаты опроса, оценки разделим на три группы:
* 9-10 баллов — «cторонники» (англ. promoters);
* 7-8 баллов — «нейтралы» (англ. passives);
* 0-6 баллов — «критики» (англ. detractors).


Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».

**Цель исследования:**

Определение текущего уровня потребительской лояльности или NPS среди клиентов из России



***На дашборд :*** https://public.tableau.com/app/profile/igor.plekhov/viz/Project2_16795330095340/ProjectDashboard
    
    
***На яндекс диск:***https://disk.yandex.ru/d/MbuML_kGjgaS0w
    
    

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_platform
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
        WITH
        
        u AS (SELECT *
              FROM user),
                 
        loc AS (SELECT location_id,
                       country,
                       city
                FROM location),         
                 
        a AS (SELECT SUBSTR(title,3,10) AS  age_segment,
                     age_gr_id
              FROM age_segment),
                
                
        tr AS (SELECT SUBSTR(title,3,10) AS traffic_segment,
                      tr_gr_id
               FROM traffic_segment),
            
        lt AS (SELECT SUBSTR(title,3,10) AS  lifetime_segment,
                      lt_gr_id
               FROM lifetime_segment)
        
        SELECT u.user_id,
               u.lt_day,
               u.age,
               u.gender_segment,
               u.os_name,
               u.cpe_type_name,
               loc.country,
               loc.city,
               a.age_segment,
               tr.traffic_segment,
               lt.lifetime_segment,
               u.nps_score,
               CASE
                   WHEN u.nps_score >= 9 THEN 'cторонники'
                   WHEN u.nps_score < 9 AND u.nps_score >=7  THEN 'нейтралы'
                   WHEN u.nps_score <= 6 THEN 'критики'
               END AS nps_group
               
               

        FROM u
        JOIN loc ON u.location_id = loc.location_id
        JOIN a ON u.age_gr_id = a.age_gr_id
        JOIN tr ON u.tr_gr_id = tr.tr_gr_id
        JOIN lt ON u.lt_gr_id = lt.lt_gr_id                 
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(10)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,45-54,1-5,36+,10,cторонники
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,45-54,1-5,36+,10,cторонники
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,55-64,20-25,13-24,10,cторонники
3,A004TB,4190,44.0,1.0,IOS,SMARTPHONE,Россия,РостовнаДону,35-44,0.1-1,36+,10,cторонники
4,A004XT,1163,24.0,0.0,ANDROID,SMARTPHONE,Россия,Рязань,16-24,5-10,36+,10,cторонники
5,A005O0,5501,42.0,1.0,ANDROID,SMARTPHONE,Россия,Омск,35-44,5-10,36+,6,критики
6,A0061R,1236,45.0,0.0,ANDROID,SMARTPHONE,Россия,Уфа,45-54,10-15,36+,10,cторонники
7,A009KS,313,35.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,35-44,45-50,7-12,10,cторонники
8,A00AES,3238,36.0,1.0,ANDROID,SMARTPHONE,Россия,СанктПетербург,35-44,1-5,36+,10,cторонники
9,A00F70,4479,54.0,1.0,ANDROID,SMARTPHONE,Россия,Волгоград,45-54,15-20,36+,9,cторонники


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   age               501939 non-null  float64
 3   gender_segment    501192 non-null  float64
 4   os_name           502493 non-null  object 
 5   cpe_type_name     502493 non-null  object 
 6   country           502493 non-null  object 
 7   city              502493 non-null  object 
 8   age_segment       502493 non-null  object 
 9   traffic_segment   502493 non-null  object 
 10  lifetime_segment  502493 non-null  object 
 11  nps_score         502493 non-null  int64  
 12  nps_group         502493 non-null  object 
dtypes: float64(2), int64(2), object(9)
memory usage: 49.8+ MB


In [6]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

***На дашборд :*** https://public.tableau.com/app/profile/igor.plekhov/viz/Project2_16795330095340/ProjectDashboard 

***На яндекс диск:***https://disk.yandex.ru/d/MbuML_kGjgaS0w